In [297]:
from configparser import ConfigParser
from pathlib import Path
import os
config = ConfigParser()

p = Path(os.getcwd()) / ".." / ".testmo.ini"
config.read(p.resolve())
sections = config.sections()
print(f"Sections: {sections}")
print(f"Sections.Keys: {list(config[sections[0]].keys())}")

Sections: ['account', 'project']
Sections.Keys: ['username', 'password', 'workspace']


In [298]:
account = config["account"]

username = account["username"]
password = account["password"]
workspace = account["workspace"]

project = config["project"]

project_name = project["project_name"]
repository_num = project["repository_num"]

In [299]:
import asyncio
from playwright.async_api import async_playwright, Playwright

In [300]:
options = dict(
    headless=False,
    slow_mo=100,
    args=[
        "--start-maximized",
        "--window-size=1920,1080",
    ],
)

In [301]:
playwright = await async_playwright().start()
browser = await playwright.chromium.launch(**options)

In [288]:
# context = await browser.new_context(
#     screen=dict(
#         width=1920,
#         height=1080,
#     ),
#     viewport=dict(
#         width=1920,
#         height=1080,
#     )
# )

# page = await context.new_page()

In [302]:
page = await browser.new_page()

In [303]:
await page.goto(f"https://{workspace}.testmo.net")
print()

In [304]:
E_email = page.locator(selector="input[name='email']")
E_password = page.locator(selector="input[name='password']")

await E_email.type(username)
await E_password.type(password)

In [305]:
E_submit = page.locator(selector="button[type='submit']")

await E_submit.click()

In [306]:
await page.goto(f"http://{workspace}.testmo.net/repositories/{repository_num}")
print()

-----------------------

In [313]:
async def get_into_group(page, group_id):
    await page.goto(f"http://{workspace}.testmo.net/repositories/{repository_num}?group_id={group_id}")
    
async def get_cases(page):
    E_testcases = list()
    
    while True:
        E_cases = page.locator(selector="table[data-target='components--table.table']").locator("tbody").locator("tr[data-id]")
        cnt = await E_cases.count()
        print("[count]")
        print(cnt)
        if cnt == 0:
            break
    
        for idx in range(cnt):
            E_tr = E_cases.nth(idx)
            # id = await E_tr.get_attribute("data-id")
            E_testcases.append(E_tr)
            
        print("testcase")
        print(len(E_testcases))
            
        E_next_btn = page.locator(selector="button[data-action='click->components--pagination#doNextPage']")
        is_exist_next = await E_next_btn.is_visible()
        
        print("is_exist_next")
        print(is_exist_next)
        if not is_exist_next:
            break
            
        is_disable_to_click = await E_next_btn.is_disabled()

        print("is disable to click?")
        print(is_disable_to_click)
        if is_disable_to_click:
            break
    
        await E_next_btn.click()
        await asyncio.sleep(1)

    print("testcase")
    print(len(E_testcases))        
    return E_testcases

async def get_case_ids(E_cases):
    ids = list()
    cnt = await E_cases.count()
    for idx in range(cnt):
        E_tr = E_cases.nth(idx)
        id = await E_tr.get_attribute("data-id")
        ids.append(id)
    return ids

In [311]:
await get_into_group(page, 259)

In [312]:
await get_cases(page)

[count]
25
testcase
25
is_exist_next
True
is disable to click?
False
[count]
18
testcase
43
is_exist_next
True
is disable to click?
True
testcase
43


[<Locator frame=<Frame name= url='https://dkargo.testmo.net/repositories/5?group_id=259&pagination_current=2'> selector="table[data-target='components--table.table'] >> tbody >> tr[data-id] >> nth=0">,
 <Locator frame=<Frame name= url='https://dkargo.testmo.net/repositories/5?group_id=259&pagination_current=2'> selector="table[data-target='components--table.table'] >> tbody >> tr[data-id] >> nth=1">,
 <Locator frame=<Frame name= url='https://dkargo.testmo.net/repositories/5?group_id=259&pagination_current=2'> selector="table[data-target='components--table.table'] >> tbody >> tr[data-id] >> nth=2">,
 <Locator frame=<Frame name= url='https://dkargo.testmo.net/repositories/5?group_id=259&pagination_current=2'> selector="table[data-target='components--table.table'] >> tbody >> tr[data-id] >> nth=3">,
 <Locator frame=<Frame name= url='https://dkargo.testmo.net/repositories/5?group_id=259&pagination_current=2'> selector="table[data-target='components--table.table'] >> tbody >> tr[data-id] >>